#Coyota vs Dogs
# Import Packages
    Import Tensorflow and the Keras classes needed to construct our model.
    Import Numpy, Random, Mathplotlib
    Import os

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras import Model
import random
print('Set-up - Imports')

Set-up - Imports


# Import Train and Test Data

In [2]:
train = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Train')
test = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Test')

# Understanding the Data
    How many Dog and Coyote images are in train and test folders

In [3]:
train_dog = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Train/dog')
train_coyote = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Train/coyote')
test_dog = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Test/dog')
test_coyote = os.path.join('C:/Users/lmnop/Documents/TrainSetTest/Test/coyote')

print(f'Total Training Dog Images {len(os.listdir(train_dog))}')
print(f'Total Training Coyote Images {len(os.listdir(train_coyote))}')
print(f'Total Test Dog Images {len(os.listdir(test_dog))}')
print(f'Total Test Coyote Images {len(os.listdir(test_coyote))}')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/lmnop/Documents/TrainSetTest/Train/dog'

### Configure the matplot parameters and take a look at a few pictures

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4

pic_index = 0

### Display 8 images, 4 dogs and 4 coyotes

In [ ]:
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_cat_pix = [os.path.join(dog_train, fname) 
                for fname in dog_train_fnames[pic_index-8:pic_index]]
next_dog_pix = [os.path.join(coyote_train, fname) 
                for fname in coyote_train_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

# Creating the model 
    Resizing the images to 150 x 150, R,G,B
    We will stack 3 {convolution + relu + maxpooling} modules. 
    Our convolutions operate on 3x3 windows and our maxpooling layers operate on 2x2 windows. 
    Our first convolution extracts 16 filters, the following one extracts 32 filters, and the last one extracts 64 filters

In [ ]:
img_input = layers.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

#### Use flatten to make our model a binary classification problem and fully connect the layers and output the model

In [ ]:
x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)

In [ ]:
model.summary()

#### Train the model with the binary_crossentropy loss

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

# Data Preprocessing
    Rescaling images and training in batches of 20 using val_datagen generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train,  
        target_size=(150, 150),  
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        test,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Training 
    

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

# Visualizing Intermediate Representations

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)

cat_img_files = [os.path.join(dog_train, f) for f in dog_train_fnames]
dog_img_files = [os.path.join(coyote_train, f) for f in coyote_train_fnames]
img_path = random.choice(cat_img_files + dog_img_files)

img = load_img(img_path, target_size=(150, 150))
x = img_to_array(img) 
x = x.reshape((1,) + x.shape)

x /= 255

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model.layers]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    n_features = feature_map.shape[-1]
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

# Evaluating Accuracy and Loss for the Model

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

# Prevent Overfitting
    using data augmentation and add dropout to our model.

### Apply horizontal flip

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=128,
                                               directory=train,
                                               shuffle=True,
                                               target_size=(150, 150))

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(augmented_images)

### Apply zoom augmentation

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=128,
                                               directory=train,
                                               shuffle=True,
                                               target_size=(150, 150))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmented_images)

### Put it all together

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=128,
                                                     directory=train,
                                                     shuffle=True,
                                                     target_size=(150, 150),
                                                     class_mode='binary')

### Visualize how a single image would look five different times when passing these augmentations randomly to the dataset. 

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Create validation data generator

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=128,
                                                 directory=test,
                                                 target_size=(150, 150),
                                                 class_mode='binary')

### Creating a Network with Dropouts

In [ ]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(150, 150 ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Compile the model

In [ ]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model.summary()

### Train the model

In [ ]:
history = model_new.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

# Final Evaluating Accuracy and Loss for the Model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(15)

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()